In [17]:
# !pip install torchsummary

In [18]:
# Necessary imports
import torch
import torchvision.models as models
from torchsummary import summary
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import requests
import json
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.utils import plot_model

In [19]:
# Load pretrained ResNet50 model
model = models.resnet50(pretrained=True)

In [20]:
# Print PyTorch model summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [21]:
# Load ImageNet class labels
url = 'https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json'
response = requests.get(url)
class_idx = response.json()
idx_to_label = {int(key): value[1] for key, value in class_idx.items()}

In [22]:
# Load and preprocess the image
img_path = '/kaggle/input/cutecat/CuteCat.jpg'  # Replace with your image path
img = Image.open(img_path)

In [23]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
img_tensor = preprocess(img).unsqueeze(0)

In [25]:
# Predict
model.eval()
with torch.no_grad():
    outputs = model(img_tensor)
    _, predicted = torch.max(outputs, 1)
    class_id = predicted.item()
    class_name = idx_to_label[class_id]
    print(f'Predicted class: {class_name} ({class_id})')

Predicted class: tabby (281)


In [26]:
torch.save(model.state_dict(), 'p1.pth')

In [27]:
# Define the Keras model
input_tensor = Input(shape=(224, 224, 3))
model_keras = ResNet50(include_top=True, weights=None, input_tensor=input_tensor)

In [28]:
# Function to map PyTorch weights to Keras model
def load_weights(model_keras, state_dict):
    def set_weights(layer, weights, bias=None):
        if isinstance(layer, tf.keras.layers.Conv2D):
            weights = weights.transpose(2, 3, 1, 0)
            if bias is not None:
                layer.set_weights([weights, bias])
            else:
                bias_shape = (weights.shape[-1],)
                zero_bias = np.zeros(bias_shape)
                layer.set_weights([weights, zero_bias])
        elif isinstance(layer, tf.keras.layers.BatchNormalization):
            gamma, beta, mean, var = weights
            layer.set_weights([gamma, beta, mean, var])

    keras_layers = model_keras.layers
    i = 0

    for name, param in state_dict.items():
        if 'conv' in name and 'weight' in name:
            while i < len(keras_layers) and not isinstance(keras_layers[i], tf.keras.layers.Conv2D):
                i += 1
            if i >= len(keras_layers):
                continue
            bias = None
            if name.replace('weight', 'bias') in state_dict:
                bias = state_dict[name.replace('weight', 'bias')].numpy()
            keras_weights_shape = keras_layers[i].get_weights()[0].shape
            if keras_weights_shape != param.numpy().transpose(2, 3, 1, 0).shape:
                i += 1
                continue
            set_weights(keras_layers[i], param.numpy(), bias)
            i += 1
        elif 'bn' in name:
            while i < len(keras_layers) and not isinstance(keras_layers[i], tf.keras.layers.BatchNormalization):
                i += 1
            if i >= len(keras_layers):
                continue
            if 'weight' in name:
                gamma = param.numpy()
            elif 'bias' in name:
                beta = param.numpy()
            elif 'running_mean' in name:
                mean = param.numpy()
            elif 'running_var' in name:
                var = param.numpy()
                keras_weights_shape = keras_layers[i].get_weights()[0].shape
                if keras_weights_shape != gamma.shape:
                    i += 1
                    continue
                set_weights(keras_layers[i], (gamma, beta, mean, var))
                i += 1

In [29]:
state_dict = torch.load('p1.pth')

In [30]:
load_weights(model_keras, state_dict)

In [31]:
model_keras.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,636,712 (97.80 MB)

 Trainable params: 25,583,592 (97.59 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [32]:
# Load and preprocess the image for Keras model
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Predict using Keras model
preds = model_keras.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: [('n04162706', 'seat_belt', 0.67013586), ('n02906734', 'broom', 0.18555482), ('n03584254', 'iPod', 0.06471199)]
